In [ ]:
import sys
import os
sys.path.append(os.getcwd())

import h5py as h5
import numpy as np
import utils

from obspy.core import UTCDateTime as udt
import matplotlib.pyplot as plt

## Define the plotting function

This function expects 4 arguments:
- detections (see example at the end of this notebook)
- n_best (plot the n_best detections)
- template (see example at the end of this notebook)
- station_idx (which station we use for plotting)

In [ ]:
def plot_n_detections(detections, n_best, template, station_idx=0):
    template_waveforms = template['waveforms'][station_idx, :, :]
    detection_waveforms = detections['waveforms'][:, station_idx, :, :]
    # only select the n_best best detections
    I = np.argsort(detections['metadata']['correlation_coefficients'])[::-1]
    I = I[:n_best]
    chronological_order = np.argsort(detections['metadata']['origin_times'][I])
    I = I[chronological_order]
    # select subsets
    detection_waveforms = detection_waveforms[I, :, :]
    OT = detections['metadata']['origin_times'][I]
    CC = detections['metadata']['correlation_coefficients'][I]
    n_components = detection_waveforms.shape[1]
    # define some useful variables before starting plotting
    buffer_extracted_events = np.int32(10. * template['sampling_rate'])
    moveouts = np.array([template['moveouts_S'][station_idx],
                         template['moveouts_S'][station_idx],
                         template['moveouts_P'][station_idx]])
    duration = template['waveforms'].shape[-1]
    components = ['N', 'E', 'Z']
    # start plotting
    time = np.linspace(0., 8., duration)
    figsize = (28, 12)
    plt.figure('detections', figsize=figsize)
    plt.suptitle('Station {}'.format(template['stations'][station_idx].decode('utf-8')))
    for c in range(n_components):
        plt.subplot(n_best + 1, n_components, 1 + c)
        plt.title(components[c])
        plt.plot(time, template_waveforms[c, :], lw=0.75, color='C3', label='Template')
        plt.xlim(time.min(), time.max())
        if c == 2:
            plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0., handlelength=0.1)
        for n in range(n_best):
            plt.subplot(n_best + 1, n_components, (1 + n)*n_components + c + 1)
            idx_start = buffer_extracted_events + moveouts[c]
            idx_end = idx_start + duration
            plt.plot(time, detection_waveforms[n, c, idx_start:idx_end], lw=0.75, color='C0',
                     label=udt(OT[n])\
                             .strftime('%Y,%m,%d -- %H:%M:%S'))
            plt.xlim(time.min(), time.max())
            if c == 2:
                plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
            if c == 0:
                plt.text(0.05, 0.5, 'CC = {:.2f}'.format(CC[n]), transform=plt.gca().transAxes,
                        bbox=dict(facecolor='white', alpha=0.5))
            if n == n_best - 1:
                plt.xlabel('Time (s)')
    plt.subplots_adjust(top=0.91,
            bottom=0.075,
            left=0.06,
            right=0.885,
            hspace=0.2,
            wspace=0.2)
    plt.show()

## Load the detections

In [ ]:
# choose n_mad according to the threshold you used before
n_mad = 10
filename = 'detections_20130317_{:d}_mad'.format(n_mad)

# retrieve the extracted events
tid = 0 # since we only used one template
detections = utils.load_detections(filename, tid, path='./output/')
print('detections is a Python dictionary with categories:\n', list(detections.keys()))
print('\n')
print('metadata is also a Python dictionary:\n', list(detections['metadata'].keys()))

## Load the template

In [ ]:
template = utils.load_template('template.h5', path='./output/')

## You are ready to play with our plotting function!

Choose the number of detections you want to plot, and call the function. See for example:

In [ ]:
# tune some plotting parameters
font = {'family': 'serif', 
        'size': 16}
plt.rc('font', **font)

In [ ]:
n_best = 20
plot_n_detections(detections, n_best, template, station_idx=2)